In [7]:
import torch
from torch import nn

class SMRNN(nn.Module):
    def __init__(self,state_size,input_size,outputs) -> None:
        super().__init__()
        self.state = torch.zeros(state_size)
        self.ln1 = nn.Linear(input_size+state_size,state_size + outputs)
    
    def forward(self,inputs):
        y = torch.cat((self.state,inputs))
        x = self.ln1(y)
        self.state = x[:self.state.shape[0]]
        return x[self.state.shape[0]:]



In [10]:
import torch.nn.functional as F
from torch import optim

a = SMRNN(3,3,3)
opt = optim.SGD(a.parameters(),lr=0.01)
loss_func = F.mse_loss
prev_inpt = torch.rand(3)
for i in range(100):
    inpt = torch.rand(3)
    y = a.forward(inpt)
    loss = loss_func(y,3*inpt+prev_inpt/2)
    print(loss)
    loss.backward(retain_graph=True)
    opt.step()
    opt.zero_grad()
    prev_inpt = inpt

tensor([0.0000, 0.0000, 0.0000, 0.6891, 0.3421, 0.6232])
tensor(4.7219, grad_fn=<MseLossBackward0>)
tensor([ 0.1198, -0.5073,  0.0751,  0.2331,  0.3913,  0.3226],
       grad_fn=<CatBackward0>)
tensor(2.0935, grad_fn=<MseLossBackward0>)


RuntimeError: Trying to backward through the graph a second time (or directly access saved tensors after they have already been freed). Saved intermediate values of the graph are freed when you call .backward() or autograd.grad(). Specify retain_graph=True if you need to backward through the graph a second time or if you need to access saved tensors after calling backward.